In [3]:
import sys; print(sys.version); 
import torch; print(torch.__version__);
import core
from core import *

3.11.0 (main, Jan 16 2023, 14:19:54) [GCC 11.2.0]
2.0.0+cu117


## x.1 使用DataModule子类构造数据

### x.1.1 Manual

In [4]:
# 构造data
class SyntheticRegressionData(DataModule):  #@save
    """Synthetic data for linear regression."""
    def __init__(self, w, b, noise=0.01, num_train=1000, num_val=1000,
                 batch_size=32):
        super().__init__()
        self.save_hyperparameters()
        n = num_train + num_val
        self.X = torch.randn(n, len(w))
        noise = torch.randn(n, 1) * noise
        self.y = torch.matmul(self.X, w.reshape((-1, 1))) + b + noise

查看数据.

最终我们生成了两千对数据（其中1k是训练集，1k是测试集）；每一对数据对中，输入量有两个features，输出量有一个label。

In [5]:
data = SyntheticRegressionData(w=torch.tensor([2, -3.4]), b=4.2)
print(data.X.shape, data.y.shape)
print('features:', data.X[0],'\nlabel:', data.y[0])

torch.Size([2000, 2]) torch.Size([2000, 1])
features: tensor([-2.3020,  1.0766]) 
label: tensor([-4.0442])


模仿Pytorch的dataloader，使用生成器写了个loader，每次运行到yield停住。但其实更加推荐pytorch内置的dataloader，在后续我们也会使用pytorch自带的loader。

有意思的代码在于以下几点：

- `random.shuffle(indices)` indices是创建的[0: 1000]共计1000个元素的list，使用`random.shuffle(indices)`将这个list中的元素随机打乱，这也是为什么随机种子中需要设置random的seed的原因。
- `torch.tensor(indices[i: i+self.batch_size])` 取出来BatchSize个元素，我们对类型为torch.Tensor的二维张量X使用X[2][3]是取第一维度为2，第二维度为3的元素；而用X[[2, 3]]则是取第一个维度中，为2，3的元素，这个是python的语法，下面两句话是一个意思：

`X[[1, 2, 3]][:] == X[1:4][:]`

- `yeild` 相当于return

In [6]:
@add_to_class(SyntheticRegressionData)
def get_dataloader(self, train):
    if train:
        indices = list(range(0, self.num_train))
        # The examples are read in random order
        random.shuffle(indices)
    else:
        indices = list(range(self.num_train, self.num_train+self.num_val))
    for i in range(0, len(indices), self.batch_size):
        batch_indices = torch.tensor(indices[i: i+self.batch_size])
        yield self.X[batch_indices], self.y[batch_indices]

查看loader中的一对数据：输入特征和对应的标签

In [7]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


### x.1.2 Concise

使用了torch.utils.data.TensorDataset, torch.utils.data.Dataset和torch.utils.data.DataLoader

本质上就是使用一些python语法，最终将输入和输出配对成Dataset类的对象，再传给DataLoader。

In [8]:
from core import *
@add_to_class(DataModule)  
def get_tensorloader(self, tensors, train, indices=slice(0, None)):
    tensors = tuple(a[indices] for a in tensors)
    dataset = torch.utils.data.TensorDataset(*tensors)
    return torch.utils.data.DataLoader(dataset, self.batch_size,
                                       shuffle=train)

@add_to_class(SyntheticRegressionData)  
def get_dataloader(self, train):
    i = slice(0, self.num_train) if train else slice(self.num_train, None)
    return self.get_tensorloader((self.X, self.y), train, i)

测试一下数据是否生成

In [9]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


使用len()调用\_\_len\_\_ 查询长度

In [10]:
len(data.train_dataloader())

32

在学习完本章后，我们大致上会使用简明方法即dataset和dataloader方法，而不再自己使用生成器yield。